<a href="https://colab.research.google.com/github/andrey101010/ds-challenge-S-and-S/blob/main/S_and_S_lidar_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# If the libraries in this cell are not installed, please uncomment and install them.
# !pip install open3d 
# !pip install laspy
# !pip install numpy

# Point Cloud Ground Segmentation

This notebook is using as an input a Lidar generated file. In this case it is a 3D map of a landscape.
The goal here is to performed image segmentation where the ground is separated from the rest of the image.
The idea here is to read out the Z loaction of the points (Z location is the hight of the point) and then create a plane, which is used as a threshold for ground and non-ground objects

In [ ]:
import laspy
import numpy as np
import open3d as o3d

In [ ]:
# to read the file the laspy library is used
ddd = laspy.read("C:/Users/Jakob/Desktop/OneDrive_2_2/lidar_challenge/input.las")

In [ ]:
#  Which properties does the ddd object have?
print(dir(ddd))

In [ ]:
list(ddd.point_format.dimension_names)

In [ ]:
# There is a possibility that the points in the cloud are alredy classified. Most lidar data label 2 as ground.
# For further information visit the website:
# https://desktop.arcgis.com/de/arcmap/10.3/manage-data/las-dataset/lidar-point-classification.htm
set(list(ddd.classification))

In [ ]:
# With this command a point cloud object is created and can be visualised later.
point_cloud = np.stack([ddd.X, ddd.Y, ddd.Z], axis = 0).transpose((1, 0))

In [ ]:
point_cloud.shape

In [ ]:
type(point_cloud)

In [ ]:
# 3D coordinates are used for matplotlib. However, due to performance reasons this action is commented out although 
# point cloud visulisation is also possible
X = point_cloud[:, 0]
Y = point_cloud[:, 1]
Z = point_cloud[:, 2]

%timeit
# fig = plt.figure(figsize=(10,10))
# ax = plt.axes(projection = '3d')
# ax.scatter(X, Y, Z, c=Z, cmap='viridis', linewidth=.5);

In [ ]:
# this is the lowest point in the dataset in Z
Z.min()

In [ ]:
# average hight in Z
Z.mean()

In [ ]:
# in this cell you can adjust the hight of the plane in the point cloud. As seen in the upper cells the lowest point is around 4500 
# the mean value is at 5500. For the presentation 3 differents hights were choosen.
lower_bound = 5000
middle = 5400
upper_bound = 5900

Z_threshold = np.full(len(Z), lower_bound) # The second arguments is for the hight of the plane. Change it to see the different outcomes!

In [ ]:
# Plane with only 1 Z value and the same dimensions as the original image
point_cloud_Z_threshold = np.stack([ddd.X, ddd.Y, Z_threshold], axis = 0).transpose((1, 0))

In [ ]:
# top view of the image without any changes. This function shows the unproccessed data.
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)
o3d.visualization.draw_geometries([pcd])

In [ ]:
# with shift and number 2, 3, 5 it is possible to select the RGB channels. While with the key 9 grayscale is acheived
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)
o3d.visualization.draw_geometries([pcd])

In [ ]:
# This is the final product. The image is generated with two datasets.

pcd_fixed_Z = o3d.geometry.PointCloud()
pcd_fixed_Z.points = o3d.utility.Vector3dVector(point_cloud_Z_threshold)
pcd_fixed_Z.paint_uniform_color([0.2, 0.3, 0.5]) # plane color change
o3d.visualization.draw_geometries([pcd, pcd_fixed_Z])

In [ ]:
# The open3d library has a function which detects planes in an image. The less the data points scatter the better this function works.
 
plane_model, inliers = pcd.segment_plane(distance_threshold=400,
                                         ransac_n=7,
                                         num_iterations=10)

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([0.7, 0, 0.3])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])